In [1]:
import os
import json
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
# Set seed
np.random.seed(42)

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


True

In [141]:
project_name = "WattCast"

# Initialize your project
api = wandb.Api()
runs = api.runs(project_name)

name_id_dict = {}

for run_ in runs:
    l = run_.name.split('_')[:-1]
    l.insert(2, 'in')
    n = '_'.join(l)
    # remove the _ around in
    n = n.replace('_in_', 'in')
    name_id_dict[n] = run_.id

del name_id_dict['results_synthesis']

KeyError: 'results_synthesis'

In [144]:
key_list = list(name_id_dict.keys())
for key in key_list:
    if key[0] not in ['1', '2', '3', '4', '5']:
        del name_id_dict[key]

In [ ]:
run = wandb.init(project='WattCast', name='results_synthesis')


In [147]:

for name, id in name_id_dict.items():

    try:
        run_path = f'wattcast/WattCast/run-{id}-Errormetricsfor{name}:v0'
        artifact = run.use_artifact(f'{run_path}', type='run_table')
        artifact_dir = artifact.download()
        with open(os.path.join(artifact_dir, os.listdir(artifact_dir)[0])) as f:
            data = json.load(f)
        df_metrics = pd.DataFrame(data['data'], columns=data['columns'])

        df = df_metrics[['horizon_in_hours', 'rmse_skill_score', 'season', 'model']]

        # Define colors and shapes based on the season and model
        season2color = {'Summer': 'orange', 'Winter': 'blue'}
        color2season = {v: k for k, v in season2color.items()}
        model2shape = {
            'RandomForest': 'circle',
            'LightGBMModel': 'square',
            'XGBModel': 'diamond',
            'BlockRNNModel': 'cross',
            'NBEATSModel': 'triangle-up',
            'LinearRegressionModel': 'star'
        }
        shape2model = {v: k for k, v in model2shape.items()}




        # Map the colors and shapes to the DataFrame
        df['color'] = df['season'].map(color_map)
        df['shape'] = df['model'].map(shape_map)

        # Create the scatter plot
        fig = px.scatter(
            df,
            x='horizon_in_hours',
            y='rmse_skill_score',
            color='color',
            symbol='shape',
            symbol_sequence= list(shape_map.values()),
            hover_data=['model'],
            title=f'RMSE Skill Score Compared to 24-Persistence over Forecast Horizon - {name}',
            labels={'rmse_skill_score': 'RMSE Skill Score', 'horizon_in_hours': 'Forecast Horizon (hours)'}

                
        )

        # set the x ticks and rename the legend
        fig.update_layout(
            xaxis=dict(
                tickmode='array',
                tickvals=[1, 4, 8, 24, 48],
            ),
            legend_title_text='Model',
        )

        for i in range(len(fig.data)):
            current_name = fig.data[i]['name']
            color, shape = current_name.split(', ')
            fig.data[i]['name'] = f'{shape2model[shape]} ({color2season[color]})'
            fig.data[i]['legendgroup'] = f'{shape2model[shape]}'

        # Update the figure size
        fig.update_layout(width=700, height=600)
        
        fig.show()

    except:
        print(f'Error for {name}')

Error for 3_villageinvillage_0


wandb:   1 of 1 files downloaded.  


wandb:   1 of 1 files downloaded.  
